1. Introduction - Get the Data

In [1]:
# Import Dependancies

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder

#Define Filepath
path = "C:/Users/Rahul/OneDrive/Desktop/Data Analyst/Projects/Grocery_Store_Sales/Store Sales/"

#Obtain all the data
store_data = pd.read_csv( path + "stores.csv")
train_data = pd.read_csv( path + "train.csv")
test_data = pd.read_csv( path + "test.csv")
oil_price = pd.read_csv( path + "oil.csv")
holidays = pd.read_csv( path + "holidays_events.csv")
submission = pd.read_csv( path + "sample_submission.csv")
transactions = pd.read_csv(path + "transactions.csv")

In [2]:
# Overview of the Training Dataset

train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/2013,1,AUTOMOTIVE,0.000,0
1,1,1/1/2013,1,BABY CARE,0.000,0
2,2,1/1/2013,1,BEAUTY,0.000,0
3,3,1/1/2013,1,BEVERAGES,0.000,0
4,4,1/1/2013,1,BOOKS,0.000,0
...,...,...,...,...,...,...
1048570,1048570,8/13/2014,3,POULTRY,974.098,1
1048571,1048571,8/13/2014,3,PREPARED FOODS,324.293,0
1048572,1048572,8/13/2014,3,PRODUCE,10.000,1
1048573,1048573,8/13/2014,3,SCHOOL AND OFFICE SUPPLIES,0.000,0


In [3]:
# Overview of the Testing Dataset

test_data

,id,date,store_nbr,family,onpromotion
0,3000888,8/16/2017,1,AUTOMOTIVE,0
1,3000889,8/16/2017,1,BABY CARE,0
2,3000890,8/16/2017,1,BEAUTY,2
3,3000891,8/16/2017,1,BEVERAGES,20
4,3000892,8/16/2017,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,8/31/2017,9,POULTRY,1
28508,3029396,8/31/2017,9,PREPARED FOODS,0
28509,3029397,8/31/2017,9,PRODUCE,1
28510,3029398,8/31/2017,9,SCHOOL AND OFFICE SUPPLIES,9


2. Data Transformation - Fix datatypes, remove duplicate columns and those with more categorical values., including Data Encoding aimed at removing categorical variables, which can be a hinderence when adpoting ML Algorithms. This will allow us to streamline our data models, reduce the features in this dataset.

In [4]:
# Obtain total unique values in each column.
holidays.nunique()

date           312
type             6
locale           3
locale_name     24
description    103
transferred      2
dtype: int64

In [5]:
# Transform the Holidays Dataset to allow for easy use in ML models

#Dropping the column as it has over 100 unique values.
holidays.drop(columns=['description'], inplace = True)

#Encoding the categorical columns

# Initialize the LabelEncoder

label_encoder = LabelEncoder()

holidays['Holiday_type_encoded'] = label_encoder.fit_transform(holidays['type'])
holidays['Holiday_locale_encoded'] = label_encoder.fit_transform(holidays['locale'])
holidays['Transferred_Holiday_encoded'] = label_encoder.fit_transform(holidays['transferred'])
holidays['Region_encoded'] = label_encoder.fit_transform(holidays['locale_name'])

holidays.drop(columns=['type','locale','locale_name','transferred'], inplace= True)
holidays

,date,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded
0,3/2/2012,3,0,0,15
1,4/1/2012,3,2,0,2
2,4/12/2012,3,0,0,3
3,4/14/2012,3,0,0,12
4,4/21/2012,3,0,0,19
...,...,...,...,...,...
345,12/22/2017,0,1,0,4
346,12/23/2017,0,1,0,4
347,12/24/2017,0,1,0,4
348,12/25/2017,3,1,0,4


In [6]:
#Encoding the categorical columns for the Store Dataset

store_data['City_encoded']=label_encoder.fit_transform(store_data['city'])
store_data['State_encoded']=label_encoder.fit_transform(store_data['state'])
store_data['Store_type_encoded']=label_encoder.fit_transform(store_data['type'])

store_data.drop(columns = ['city','state','type'], inplace = True)

In [7]:
#Preprocessing the data

train_data['Family_Encoded']=label_encoder.fit_transform(train_data['family'])
test_data['Family_Encoded']=label_encoder.fit_transform(test_data['family'])

#Drop categorical columns

test_data.drop(columns = ['family'], inplace = True)
train_data.drop(columns = ['family'], inplace = True)

In [8]:
#Split the Oil prices Dataset into test and train. This will allow us to merge these datasets easily.

oil_price_test = oil_price[oil_price['date']>=test_data['date'][0]]
oil_price_train = oil_price[oil_price['date']<train_data['date'].iloc[-1]]

In [9]:
#Review the Oil Prices Train Dataset
oil_price_train

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [10]:
#Review the Transaction Dataset
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [11]:
# Ensure 'date' columns are in datetime64[ns] format in all DataFrames
holidays['date'] = pd.to_datetime(holidays['date'])
train_data['date'] = pd.to_datetime(train_data['date'])
oil_price_train['date'] = pd.to_datetime(oil_price['date'])
oil_price_test['date'] = pd.to_datetime(oil_price['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

3. Merging Datasets - Merge all the tables to obtain a training and testing dataset. Because train data has more than one occurence of date, we use right join to merge to two tables on date, in an attempt to force all the data in the training dataset to be included.

In [12]:
# Merge the Datasets
train_final = pd.merge(holidays, train_data, on='date', how = 'right')
train_final = pd.merge(train_final, store_data, on='store_nbr', how='inner')
train_final = pd.merge(train_final, oil_price_train, on='date')
train_final = pd.merge(train_final, transactions, on=['date', 'store_nbr'], how='left')


In [13]:
# Review the training dataset

train_final

,date,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,sales,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions
0,2013-01-01,3.0,1.0,0.0,4.0,0,1,0.000,0,0,13,18,12,3,NaN,NaN
1,2013-01-01,3.0,1.0,0.0,4.0,1,1,0.000,0,1,13,18,12,3,NaN,NaN
2,2013-01-01,3.0,1.0,0.0,4.0,2,1,0.000,0,2,13,18,12,3,NaN,NaN
3,2013-01-01,3.0,1.0,0.0,4.0,3,1,0.000,0,3,13,18,12,3,NaN,NaN
4,2013-01-01,3.0,1.0,0.0,4.0,4,1,0.000,0,4,13,18,12,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761668,2014-08-13,NaN,NaN,NaN,NaN,1048570,3,974.098,1,26,8,18,12,3,97.57,2991.0
761669,2014-08-13,NaN,NaN,NaN,NaN,1048571,3,324.293,0,27,8,18,12,3,97.57,2991.0
761670,2014-08-13,NaN,NaN,NaN,NaN,1048572,3,10.000,1,28,8,18,12,3,97.57,2991.0
761671,2014-08-13,NaN,NaN,NaN,NaN,1048573,3,0.000,0,29,8,18,12,3,97.57,2991.0


In [14]:
# Convert date string feature to timestamps
train_final['date'] = pd.to_datetime(train_final['date'])
train_final['Day']=train_final['date'].dt.day
train_final['Month']=train_final['date'].dt.month
train_final['Year']=train_final['date'].dt.year
train_final['Day of the week']=train_final['date'].dt.dayofweek
train_final['Day of the year']=train_final['date'].dt.dayofyear

#Prepare the train data
train_final.drop(columns = ['date'], inplace = True)
train_final.fillna(0,inplace = True)

In [15]:
train_final

,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,sales,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions,Day,Month,Year,Day of the week,Day of the year
0,3.0,1.0,0.0,4.0,0,1,0.000,0,0,13,18,12,3,0.00,0.0,1,1,2013,1,1
1,3.0,1.0,0.0,4.0,1,1,0.000,0,1,13,18,12,3,0.00,0.0,1,1,2013,1,1
2,3.0,1.0,0.0,4.0,2,1,0.000,0,2,13,18,12,3,0.00,0.0,1,1,2013,1,1
3,3.0,1.0,0.0,4.0,3,1,0.000,0,3,13,18,12,3,0.00,0.0,1,1,2013,1,1
4,3.0,1.0,0.0,4.0,4,1,0.000,0,4,13,18,12,3,0.00,0.0,1,1,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761668,0.0,0.0,0.0,0.0,1048570,3,974.098,1,26,8,18,12,3,97.57,2991.0,13,8,2014,2,225
761669,0.0,0.0,0.0,0.0,1048571,3,324.293,0,27,8,18,12,3,97.57,2991.0,13,8,2014,2,225
761670,0.0,0.0,0.0,0.0,1048572,3,10.000,1,28,8,18,12,3,97.57,2991.0,13,8,2014,2,225
761671,0.0,0.0,0.0,0.0,1048573,3,0.000,0,29,8,18,12,3,97.57,2991.0,13,8,2014,2,225


In [16]:
#Explore all the datatypes

train_final.dtypes

Holiday_type_encoded           float64
Holiday_locale_encoded         float64
Transferred_Holiday_encoded    float64
Region_encoded                 float64
id                               int64
store_nbr                        int64
sales                          float64
onpromotion                      int64
Family_Encoded                   int64
cluster                          int64
City_encoded                     int64
State_encoded                    int64
Store_type_encoded               int64
dcoilwtico                     float64
transactions                   float64
Day                              int32
Month                            int32
Year                             int32
Day of the week                  int32
Day of the year                  int32
dtype: object

In [17]:
# Ensure 'date' columns are in datetime64[ns] format in all DataFrames

test_data['date'] = pd.to_datetime(test_data['date'])

#Merge the Holidays, Store_data, Oil_prices, and test_data

test_final = pd.merge(holidays, test_data, on = 'date', how = 'outer')
test_final = pd.merge(test_final, store_data, on='store_nbr',how = 'inner')
test_final = pd.merge(test_final, oil_price_test, on = 'date', how = 'left')
test_final = pd.merge(test_final, transactions, on = ['date','store_nbr'], how = 'left')

In [18]:
# Overview of the testing dataset
test_final

,date,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions
0,2017-08-16,NaN,NaN,NaN,NaN,3000888.0,1.0,0.0,0.0,13,18,12,3,NaN,NaN
1,2017-08-16,NaN,NaN,NaN,NaN,3000889.0,1.0,0.0,1.0,13,18,12,3,NaN,NaN
2,2017-08-16,NaN,NaN,NaN,NaN,3000890.0,1.0,2.0,2.0,13,18,12,3,NaN,NaN
3,2017-08-16,NaN,NaN,NaN,NaN,3000891.0,1.0,20.0,3.0,13,18,12,3,NaN,NaN
4,2017-08-16,NaN,NaN,NaN,NaN,3000892.0,1.0,0.0,4.0,13,18,12,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,NaN,NaN,NaN,NaN,3029395.0,9.0,1.0,26.0,6,18,12,1,NaN,NaN
28508,2017-08-31,NaN,NaN,NaN,NaN,3029396.0,9.0,0.0,27.0,6,18,12,1,NaN,NaN
28509,2017-08-31,NaN,NaN,NaN,NaN,3029397.0,9.0,1.0,28.0,6,18,12,1,NaN,NaN
28510,2017-08-31,NaN,NaN,NaN,NaN,3029398.0,9.0,9.0,29.0,6,18,12,1,NaN,NaN


In [19]:

# Convert date string feature to timestamps
test_final['date'] = pd.to_datetime(test_final['date'], format='mixed')
test_final['Day']=test_final['date'].dt.day
test_final['Month']= test_final['date'].dt.month
test_final['Year']=test_final['date'].dt.year
test_final['Day of the week']=test_final['date'].dt.dayofweek
test_final['Day of the year']=test_final['date'].dt.dayofyear

#Change Datatype of oil prices
test_final['dcoilwtico'] = test_final['dcoilwtico'].astype(float)

#Prepare the test data
test_final.drop(columns = ['date'], inplace = True)
test_final.fillna(0, inplace= True)
test_final['sales']=np.nan

test_final


,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions,Day,Month,Year,Day of the week,Day of the year,sales
0,0.0,0.0,0.0,0.0,3000888.0,1.0,0.0,0.0,13,18,12,3,0.0,0.0,16,8,2017,2,228,NaN
1,0.0,0.0,0.0,0.0,3000889.0,1.0,0.0,1.0,13,18,12,3,0.0,0.0,16,8,2017,2,228,NaN
2,0.0,0.0,0.0,0.0,3000890.0,1.0,2.0,2.0,13,18,12,3,0.0,0.0,16,8,2017,2,228,NaN
3,0.0,0.0,0.0,0.0,3000891.0,1.0,20.0,3.0,13,18,12,3,0.0,0.0,16,8,2017,2,228,NaN
4,0.0,0.0,0.0,0.0,3000892.0,1.0,0.0,4.0,13,18,12,3,0.0,0.0,16,8,2017,2,228,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,0.0,0.0,0.0,0.0,3029395.0,9.0,1.0,26.0,6,18,12,1,0.0,0.0,31,8,2017,3,243,NaN
28508,0.0,0.0,0.0,0.0,3029396.0,9.0,0.0,27.0,6,18,12,1,0.0,0.0,31,8,2017,3,243,NaN
28509,0.0,0.0,0.0,0.0,3029397.0,9.0,1.0,28.0,6,18,12,1,0.0,0.0,31,8,2017,3,243,NaN
28510,0.0,0.0,0.0,0.0,3029398.0,9.0,9.0,29.0,6,18,12,1,0.0,0.0,31,8,2017,3,243,NaN


In [20]:
#Unique values in the training set
train_final.nunique()

Holiday_type_encoded                4
Holiday_locale_encoded              3
Transferred_Holiday_encoded         2
Region_encoded                     20
id                             749199
store_nbr                          54
sales                           94851
onpromotion                       159
Family_Encoded                     31
cluster                            17
City_encoded                       22
State_encoded                      16
Store_type_encoded                  5
dcoilwtico                        360
transactions                     3468
Day                                31
Month                              12
Year                                2
Day of the week                     5
Day of the year                   292
dtype: int64

4. Split the data into training and testing sets

In [21]:
test_final.columns

Index(['Holiday_type_encoded', 'Holiday_locale_encoded',
       'Transferred_Holiday_encoded', 'Region_encoded', 'id', 'store_nbr',
       'onpromotion', 'Family_Encoded', 'cluster', 'City_encoded',
       'State_encoded', 'Store_type_encoded', 'dcoilwtico', 'transactions',
       'Day', 'Month', 'Year', 'Day of the week', 'Day of the year', 'sales'],
      dtype='object')

In [22]:
train_final.columns

Index(['Holiday_type_encoded', 'Holiday_locale_encoded',
       'Transferred_Holiday_encoded', 'Region_encoded', 'id', 'store_nbr',
       'sales', 'onpromotion', 'Family_Encoded', 'cluster', 'City_encoded',
       'State_encoded', 'Store_type_encoded', 'dcoilwtico', 'transactions',
       'Day', 'Month', 'Year', 'Day of the week', 'Day of the year'],
      dtype='object')

In [23]:
#save the dataframe
train_final.to_csv(path +"Final_Data.csv", index = False)

In [24]:
# Set the test and train data

X_train, X_test, y_train, y_test = train_test_split(train_final.drop(columns=['sales']), train_final['sales'], test_size=0.25, random_state=4)

In [25]:
X_train.columns

Index(['Holiday_type_encoded', 'Holiday_locale_encoded',
       'Transferred_Holiday_encoded', 'Region_encoded', 'id', 'store_nbr',
       'onpromotion', 'Family_Encoded', 'cluster', 'City_encoded',
       'State_encoded', 'Store_type_encoded', 'dcoilwtico', 'transactions',
       'Day', 'Month', 'Year', 'Day of the week', 'Day of the year'],
      dtype='object')

In [26]:
X_train.dtypes

Holiday_type_encoded           float64
Holiday_locale_encoded         float64
Transferred_Holiday_encoded    float64
Region_encoded                 float64
id                               int64
store_nbr                        int64
onpromotion                      int64
Family_Encoded                   int64
cluster                          int64
City_encoded                     int64
State_encoded                    int64
Store_type_encoded               int64
dcoilwtico                     float64
transactions                   float64
Day                              int32
Month                            int32
Year                             int32
Day of the week                  int32
Day of the year                  int32
dtype: object

5. LightGBM - LightGBM is one of the fastest and most efficient gradient boosting algorithms available, especially for large-scale datasets. Its ability to handle large data, native support for categorical features, and faster training time makes it a preferred choice for machine learning tasks involving structured data. In this case as we are employing other models as well, the data has been encoded.

In [27]:
#Using lightgbm 
import lightgbm as lgb

# Define parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 62,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'force_row_wise':True
}


# Convert training data to a LightGBM Dataset
train_data1 = lgb.Dataset(X_train, label=y_train)

# Train the LightGBM model
model = lgb.train(params, train_data1, num_boost_round=175)

# Prediction
y_pred1 = model.predict(X_test)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred1, 0)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 1314
[LightGBM] [Info] Number of data points in the train set: 571254, number of used features: 19
[LightGBM] [Info] Start training from score 216.389448


In [28]:
# Calculate RMSLE

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)
    
print("Root Mean Squared Logarithmic Error:", rmsle)
print("Light GBM predicted " + str(len(y_pred1[y_pred1<0])/len(y_pred1)* 100) + "% negative resutls")

Root Mean Squared Logarithmic Error: 2.8821053839482578
Light GBM predicted 26.618667254843263% negative resutls


6. XGBoost - XGBoost is a powerful, scalable, and efficient machine learning algorithm, particularly suitable for structured/tabular data. Its speed, flexibility, and performance have made it a go-to ML model for prediction. The combination of regularization, handling of missing data, and support for large-scale data makes XGBoost one of the most widely used and respected algorithms.

In [29]:
#Using XGBoost
import xgboost as xgb


# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 20,
    'learning_rate': 0.1,
    'subsample': 1
}

# Define XGBoost DMatrix
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test, label=y_test)

# Train the XGBoost model
model2 = xgb.train(params, train_dmatrix, num_boost_round=200, evals=[(test_dmatrix, 'test')], early_stopping_rounds=5)

# Prediction
y_pred2 = model2.predict(test_dmatrix)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred2, 0)

[0]	test-rmse:663.92487
[1]	test-rmse:634.77841
[2]	test-rmse:611.01327
[3]	test-rmse:591.80015
[4]	test-rmse:577.69878
[5]	test-rmse:566.81079
[6]	test-rmse:558.55929
[7]	test-rmse:553.45137
[8]	test-rmse:550.24664
[9]	test-rmse:549.14632
[10]	test-rmse:549.48942
[11]	test-rmse:550.50632
[12]	test-rmse:552.39842
[13]	test-rmse:554.92905
[14]	test-rmse:558.35057


In [30]:
# Calculate RMSLE for XGBoost

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)
    
print("Root Mean Squared Logarithmic Error:", rmsle)
print("XGBoost predicted " + str((len(y_pred2[y_pred2<0])/len(y_pred2)* 100)) + "% negative resutls")

Root Mean Squared Logarithmic Error: 8.455607091496299
XGBoost predicted 0.0005251576785929975% negative resutls


7. Random Forrest - The Random Forest Regressor is an ensemble technique that uses a collection of decision trees to make predictions for regression tasks. The idea is to combine the predictions of several trees to improve the accuracy and robustness of the model. It works by creating many decision trees during training and averaging their outputs for predictions in regression problems. In essence, it’s a bagging technique, where multiple decision trees are trained on different subsets of the data, and the predictions from each tree are combined to form the final output.



In [31]:
#Using Random Forrest Refressor
from sklearn.ensemble import RandomForestRegressor

# Model training with lower runtime
model3 = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1, random_state=2)
model3.fit(X_train, y_train)

# Model evaluation
y_pred3 = model3.predict(X_test)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred3, 0)



In [32]:
# Calculate RMSLE

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)

print("Root Mean Squared Logarithmic Error:", rmsle)
print("Random Forrest Regressors predicted " + str(len(y_pred3[y_pred3<0])/len(y_pred3)* 100) + "% negative resutls")

Root Mean Squared Logarithmic Error: 0.7425766252164062
Random Forrest Regressors predicted 0.0% negative resutls


8. Final Step - Predicting Sales - The best results is obtained from Random Forrest Regressors. Therefore we use that ML model to obtain our store sales prediction.

In [33]:
# Define XGBoost Output DMatrix and Predict the sample set
test_dmatrix = xgb.DMatrix(data=test_final.drop(columns=['sales']), enable_categorical=True)
sample = model2.predict(test_dmatrix)

In [34]:
# Predict the submission set with random forrest regressors

sample1 = model3.predict(test_final.drop(columns = ['sales']))

In [35]:
# Prepare the submission file

submission['sales'] = sample1.round(0)
submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,26.0
4,3000892,1.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,7.0
28510,3029398,4.0


In [36]:
test_final['sales']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
28507   NaN
28508   NaN
28509   NaN
28510   NaN
28511   NaN
Name: sales, Length: 28512, dtype: float64

In [37]:
submission.to_csv(path+ "Submission_Final.csv")